In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Amapá - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)', 'Amapá - Produção de Cimento (t)'], axis=1)
data

,Ano,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Consumo de Cimento (t)
0,2003,0.711421,8.514392,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.035813e+06,356591.003430,10.883143,7.033593e+06,85.0930
1,2004,0.713006,8.446721,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.163242e+06,357271.494323,10.907898,7.078752e+06,80.0080
2,2005,0.714376,8.415811,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.284323e+06,359035.074957,10.930315,7.120175e+06,104.1970
3,2006,0.715533,8.334802,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.397333e+06,361349.457774,10.963941,7.156515e+06,115.5400
4,2007,0.716480,8.318744,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.489305e+06,362177.947424,10.970195,7.171045e+06,135.0460
5,2008,0.717218,8.281094,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.565394e+06,362905.280442,10.964829,7.173307e+06,116.3660
6,2009,0.717750,8.302950,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.619980e+06,360014.231547,10.929234,7.163325e+06,112.7290
7,2010,0.718079,8.270873,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.840356e+06,365333.788458,10.918810,7.227241e+06,151.5420
8,2011,0.718205,8.258737,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,9.053310e+06,369799.968255,10.934033,7.289555e+06,161.9700
9,2012,0.717668,14.118134,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,9.243435e+06,371290.647450,10.930025,7.338668e+06,261.1200


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,0.711421,8.514392,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.035813e+06,356591.003430,10.883143,7.033593e+06
1,0.713006,8.446721,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.163242e+06,357271.494323,10.907898,7.078752e+06
2,0.714376,8.415811,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.284323e+06,359035.074957,10.930315,7.120175e+06
3,0.715533,8.334802,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.397333e+06,361349.457774,10.963941,7.156515e+06
4,0.716480,8.318744,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.489305e+06,362177.947424,10.970195,7.171045e+06
5,0.717218,8.281094,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.565394e+06,362905.280442,10.964829,7.173307e+06
6,0.717750,8.302950,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.619980e+06,360014.231547,10.929234,7.163325e+06
7,0.718079,8.270873,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.840356e+06,365333.788458,10.918810,7.227241e+06
8,0.718205,8.258737,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,9.053310e+06,369799.968255,10.934033,7.289555e+06
9,0.717668,14.118134,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,9.243435e+06,371290.647450,10.930025,7.338668e+06


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      80.0080
1     104.1970
2     115.5400
3     135.0460
4     116.3660
5     112.7290
6     151.5420
7     161.9700
8     261.1200
9     250.5190
10    227.2430
11    198.0696
12    166.8504
13    135.0210
14    123.3470
15    120.1530
16    133.5230
17    157.6210
18    142.7390
19         NaN
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,-2.498491,-0.888884,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.651592,-0.411311,-0.183091,-1.880195
1,-1.725700,-0.907434,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.430423,-0.315776,0.264667,-1.501512
2,-1.057698,-0.915907,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.220272,-0.068185,0.670127,-1.154156
3,-0.493413,-0.938113,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.024129,0.256733,1.278344,-0.849422
4,-0.031772,-0.942515,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.864499,0.373045,1.391456,-0.727572
5,0.328298,-0.952835,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.732437,0.475156,1.294411,-0.708607
6,0.587870,-0.946844,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.637697,0.069279,0.650588,-0.792315
7,0.748015,-0.955637,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.255205,0.816097,0.462035,-0.256339
8,0.809807,-0.958964,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.114404,1.443108,0.737389,0.266205
9,0.547992,0.647212,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.444390,1.652386,0.664881,0.678053


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      80.0080
1     104.1970
2     115.5400
3     135.0460
4     116.3660
5     112.7290
6     151.5420
7     161.9700
8     261.1200
9     250.5190
10    227.2430
11    198.0696
12    166.8504
13    135.0210
14    123.3470
15    120.1530
16    133.5230
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
17,-1.683117,0.92733,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.821846,-1.025345,-1.444524,0.649078


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    157.621
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [12]:
def validation_row_splitter(df): 
    pos = df.shape[0] - 2
    return df.drop(pos), df.iloc[pos:pos + 1]

In [13]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(1, split_factor + 1):
        pos = df.shape[0] - (i*2 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [27]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 5)
    target,target_val = validation_splitter(train_target, 5)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [28]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [29]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1948816076, 2131867697, 3038382748, 1947664215, 4126652175, 34891080, 787656152, 2958440596, 4134044428, 2624371610, 1311409609, 270943428, 2656248360, 2713232116, 1178801730, 2602277949, 2023401075, 1599996491, 3268309115, 1340843117, 2206948954, 3304655323, 3197404912, 3700987511, 2853036467, 415873335, 1218272019, 2385137003, 3453198996, 2339311641, 3233054125, 2643148416, 1837441293, 2254916053, 3294125714, 1784001809, 636821081, 2615433709, 4135774362, 3645673807, 1635357797, 3678033897, 778341988, 1787123299, 2005206899, 364304901, 956499023, 1873295327, 3865144135, 398032429, 2904954895, 4203927614, 3301927773, 2839309320, 1258604721, 1386431643, 2403624104, 445878932, 2021853449, 499041622, 3346294476, 3285152444, 4182421224, 590608468, 408899885, 3224949659, 1260245267, 380202803, 1299690195, 2803695124, 2582187253, 714923674, 4082519645, 2129559107, 1987385602, 3009907029, 2949396240, 1121086581, 341148985, 94107641, 1179976110, 2803763824, 2245483975, 3578552927, 1078783798

val_loss: 190.79261779785156


Step: 83 ___________________________________________
val_loss: 103.9404525756836


Step: 84 ___________________________________________
val_loss: 96.63748168945312


Step: 85 ___________________________________________
val_loss: 271.8268737792969


Step: 86 ___________________________________________
val_loss: 685.4462280273438


Step: 87 ___________________________________________
val_loss: 9.588851928710938


Step: 88 ___________________________________________
val_loss: 538.6535034179688


Step: 89 ___________________________________________
val_loss: 34.9669189453125


Step: 90 ___________________________________________
val_loss: 9.166263580322266


Step: 91 ___________________________________________
val_loss: 188.54039001464844


Step: 92 ___________________________________________
val_loss: 41.234615325927734


Step: 93 ___________________________________________
val_loss: 2072.6376953125


Step: 94 ___________________________________________
val_

In [30]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 454ms/step - loss: 24624.4277 - val_loss: 28731.2246
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 21855.9844 - val_loss: 8395.8145
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 22191.5273 - val_loss: 44446.9062
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 19862.8145 - val_loss: 12028.6494
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 16371.9980 - val_loss: 7602.9087
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 15823.8877 - val_loss: 9074.5264
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 15571.7236 - val_loss: 8361.7881
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 15332.0518 - val_loss: 8108.9478
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 15110.4814 - val_loss: 8492.5869
Epoch 10/10000
1/1 [=============

1/1 [==============================] - 0s 15ms/step - loss: 2552.7700 - val_loss: 1871.8566
Epoch 78/10000
1/1 [==============================] - 0s 15ms/step - loss: 2552.6799 - val_loss: 1874.0400
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 2551.3086 - val_loss: 1882.1665
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 2552.2422 - val_loss: 1884.9369
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 2551.9966 - val_loss: 1895.3202
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 2553.9514 - val_loss: 1893.5254
Epoch 83/10000
1/1 [==============================] - 0s 15ms/step - loss: 2551.0662 - val_loss: 1885.0083
Epoch 84/10000
1/1 [==============================] - 0s 14ms/step - loss: 2551.2153 - val_loss: 1895.0536
Epoch 85/10000
1/1 [==============================] - 0s 14ms/step - loss: 2551.7068 - val_loss: 1870.6432
Epoch 86/10000
1/1 [==============================] 

1/1 [==============================] - 0s 14ms/step - loss: 2568.5818 - val_loss: 1829.0988
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 2554.0161 - val_loss: 1863.9435
Epoch 155/10000
1/1 [==============================] - 0s 15ms/step - loss: 2551.0886 - val_loss: 1885.4619
Epoch 156/10000
1/1 [==============================] - 0s 15ms/step - loss: 2550.8879 - val_loss: 1888.3833
Epoch 157/10000
1/1 [==============================] - 0s 15ms/step - loss: 2550.8433 - val_loss: 1881.0024
Epoch 158/10000
1/1 [==============================] - 0s 14ms/step - loss: 2552.3320 - val_loss: 1882.9150
Epoch 159/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.9631 - val_loss: 1900.3164
Epoch 160/10000
1/1 [==============================] - 0s 14ms/step - loss: 2551.1116 - val_loss: 1906.5000
Epoch 161/10000
1/1 [==============================] - 0s 15ms/step - loss: 2550.8789 - val_loss: 1896.0103
Epoch 162/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 2551.1272 - val_loss: 1875.6948
Epoch 230/10000
1/1 [==============================] - 0s 15ms/step - loss: 2551.0876 - val_loss: 1883.7499
Epoch 231/10000
1/1 [==============================] - 0s 15ms/step - loss: 2551.2546 - val_loss: 1892.3507
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.8521 - val_loss: 1876.6764
Epoch 233/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.8086 - val_loss: 1879.1425
Epoch 234/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.9153 - val_loss: 1874.6851
Epoch 235/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.8215 - val_loss: 1877.3060
Epoch 236/10000
1/1 [==============================] - 0s 14ms/step - loss: 2551.3772 - val_loss: 1892.7509
Epoch 237/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.8540 - val_loss: 1877.3315
Epoch 238/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 2550.8081 - val_loss: 1879.6498
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.7981 - val_loss: 1881.1416
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.8010 - val_loss: 1881.7362
Epoch 308/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.7571 - val_loss: 1881.1525
Epoch 309/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.8481 - val_loss: 1877.0824
Epoch 310/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.7981 - val_loss: 1882.4117
Epoch 311/10000
1/1 [==============================] - 0s 14ms/step - loss: 2550.8042 - val_loss: 1878.3424
Epoch 312/10000
1/1 [==============================] - 0s 15ms/step - loss: 2551.0237 - val_loss: 1876.3336
Epoch 313/10000
1/1 [==============================] - 0s 15ms/step - loss: 2550.7341 - val_loss: 1886.2062
Epoch 314/10000
1/1 [=======================

...

1/1 [==============================] - 0s 14ms/step - loss: 357.1147 - val_loss: 1310.1912
Epoch 992/10000
1/1 [==============================] - 0s 14ms/step - loss: 354.8896 - val_loss: 1317.0856
Epoch 993/10000
1/1 [==============================] - 0s 15ms/step - loss: 354.6004 - val_loss: 1285.7720
Epoch 994/10000
1/1 [==============================] - 0s 15ms/step - loss: 354.5504 - val_loss: 1323.2292
Epoch 995/10000
1/1 [==============================] - 0s 15ms/step - loss: 353.6111 - val_loss: 1319.1617
Epoch 996/10000
1/1 [==============================] - 0s 15ms/step - loss: 353.2145 - val_loss: 1321.3353
Epoch 997/10000
1/1 [==============================] - 0s 15ms/step - loss: 352.8675 - val_loss: 1322.8542
Epoch 998/10000
1/1 [==============================] - 0s 14ms/step - loss: 351.3492 - val_loss: 1315.6276
Epoch 999/10000
1/1 [==============================] - 0s 14ms/step - loss: 350.4246 - val_loss: 1311.6282
Epoch 1000/10000
1/1 [==============================]

1/1 [==============================] - 0s 14ms/step - loss: 323.1134 - val_loss: 1384.6803
Epoch 1068/10000
1/1 [==============================] - 0s 14ms/step - loss: 320.1236 - val_loss: 1387.8051
Epoch 1069/10000
1/1 [==============================] - 0s 14ms/step - loss: 318.1995 - val_loss: 1376.4645
Epoch 1070/10000
1/1 [==============================] - 0s 14ms/step - loss: 320.0204 - val_loss: 1407.7021
Epoch 1071/10000
1/1 [==============================] - 0s 15ms/step - loss: 317.5953 - val_loss: 1391.8667
Epoch 1072/10000
1/1 [==============================] - 0s 15ms/step - loss: 317.2649 - val_loss: 1395.5229
Epoch 1073/10000
1/1 [==============================] - 0s 15ms/step - loss: 316.8967 - val_loss: 1379.3881
Epoch 1074/10000
1/1 [==============================] - 0s 14ms/step - loss: 316.9047 - val_loss: 1400.7330
Epoch 1075/10000
1/1 [==============================] - 0s 15ms/step - loss: 316.5080 - val_loss: 1399.1010
Epoch 1076/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 304.1838 - val_loss: 1456.8783
Epoch 1144/10000
1/1 [==============================] - 0s 14ms/step - loss: 304.0488 - val_loss: 1457.9396
Epoch 1145/10000
1/1 [==============================] - 0s 15ms/step - loss: 303.9146 - val_loss: 1458.8960
Epoch 1146/10000
1/1 [==============================] - 0s 14ms/step - loss: 303.7665 - val_loss: 1460.3680
Epoch 1147/10000
1/1 [==============================] - 0s 14ms/step - loss: 303.6422 - val_loss: 1460.9999
Epoch 1148/10000
1/1 [==============================] - 0s 14ms/step - loss: 303.5191 - val_loss: 1461.7140
Epoch 1149/10000
1/1 [==============================] - 0s 14ms/step - loss: 303.3706 - val_loss: 1463.2456
Epoch 1150/10000
1/1 [==============================] - 0s 14ms/step - loss: 303.2104 - val_loss: 1466.5626
Epoch 1151/10000
1/1 [==============================] - 0s 14ms/step - loss: 303.1063 - val_loss: 1464.8055
Epoch 1152/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 296.2118 - val_loss: 1509.7518
Epoch 1220/10000
1/1 [==============================] - 0s 14ms/step - loss: 296.1490 - val_loss: 1509.7579
Epoch 1221/10000
1/1 [==============================] - 0s 14ms/step - loss: 296.1931 - val_loss: 1509.1229
Epoch 1222/10000
1/1 [==============================] - 0s 14ms/step - loss: 295.9738 - val_loss: 1513.5176
Epoch 1223/10000
1/1 [==============================] - 0s 14ms/step - loss: 295.9026 - val_loss: 1512.8989
Epoch 1224/10000
1/1 [==============================] - 0s 14ms/step - loss: 295.8235 - val_loss: 1513.7531
Epoch 1225/10000
1/1 [==============================] - 0s 14ms/step - loss: 295.7363 - val_loss: 1514.5753
Epoch 1226/10000
1/1 [==============================] - 0s 14ms/step - loss: 295.6665 - val_loss: 1514.8931
Epoch 1227/10000
1/1 [==============================] - 0s 15ms/step - loss: 295.5587 - val_loss: 1516.8818
Epoch 1228/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 291.3515 - val_loss: 1531.7860
Epoch 1296/10000
1/1 [==============================] - 0s 26ms/step - loss: 291.4846 - val_loss: 1529.6481
Epoch 1296: early stopping


In [31]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 52ms/step
[157.621](test_target) - [[146.4112]](prediction) = 11.20980615234376


In [32]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [33]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,-1.822242,1.867029,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.595141,-1.449699,-1.742940,-1.876401
1,-1.111250,1.045116,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.072139,-1.202688,-0.886900,-1.083744
2,-0.496667,0.669687,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.575192,-0.562528,-0.111729,-0.356662
3,0.022494,-0.314227,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.111371,0.277568,1.051082,0.281205
4,0.447218,-0.509271,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.266106,0.578301,1.267334,0.536260
5,0.778494,-0.966548,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.578397,0.842315,1.081801,0.575958
6,1.017308,-0.701091,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.802429,-0.207105,-0.149082,0.400740
7,1.164646,-1.090695,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.706911,1.723837,-0.509565,1.522643


train_target:


,Amapá - Consumo de Cimento (t)
0,80.008
1,104.197
2,115.540
3,135.046
4,116.366
5,112.729
6,151.542
7,161.970


test_input:


...

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
17,-1.683117,0.92733,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.821846,-1.025345,-1.444524,0.649078


test_target:


,Amapá - Consumo de Cimento (t)
17,157.621


1/1 [==============================] - 0s 34ms/step
Error: 11.20980615234376




[140.48919677734375,
 198.2721710205078,
 241.32008361816406,
 213.61676025390625,
 210.19384765625,
 194.44161987304688,
 166.61273193359375,
 161.88473510742188,
 160.27442932128906,
 146.41119384765625]

In [34]:
display(targets)
display(predictions)

[261.12,
 250.519,
 227.243,
 198.0696,
 166.85039999999998,
 135.021,
 123.347,
 120.153,
 133.523,
 157.621]

[140.48919677734375,
 198.2721710205078,
 241.32008361816406,
 213.61676025390625,
 210.19384765625,
 194.44161987304688,
 166.61273193359375,
 161.88473510742188,
 160.27442932128906,
 146.41119384765625]

In [35]:
mae = mean_absolute_error(predictions, targets)
mae

42.82246461181641

In [36]:
porcentage = mae/np.mean(targets)
porcentage

0.241461863185593